**Author**: J W Debelius<br/>
**Date**: 27 August 2015<br/>
**virtualenv**: power play

# 1. Introduction

In [ ]:
from __future__ import division

import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import statsmodels.api as sms
import statsmodels.formula.api as smf

import absloute_power.traditional as trad
import absloute_power.utils as ap

from scipy.stats import norm as Z

import americangut.power_plots as pp

import pandas as pd

from skbio.stats.power import subsample_power

from absloute_power.traditional import calc_ttest_ind

In [ ]:
from matplotlib import rcParams



# Sets up plotting parameters so that the default setting is use to Helvetica
# in plots
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Helvetica', 'Arial']
rcParams['text.usetex'] = True

% matplotlib inline

We'll examine five common, simple statistical models where the power equation has been defined and employed. The emperical power implementations are based on the version described in [ref].

We will examine statistical power for three tests dealing with normal distributions:

* Case I t test
* Case II (independent) t test
* One way Analysis of Variance (ANOVA)
* Pearson's R
* Chi-Square

We will perform 100 simulations of statistical power, comparing the power between 5 and 50 counts, with 5 observation intervals between. We'll set the critical value at 0.05.

In [ ]:
num_rounds = 100
counts = np.arange(5, 65, 10)
alpha = 0.05
distributions = {}

We'll also set a random seed, so that the results we get out are consistent.

# 2. Case I Test

A case I t-test checks if an observation is drawn from a sample. We are testing the alternatively hypotheses,

$\begin{matrix}
\textbf{H}_{0} & x = \bar{x} \\
\textbf{H}_{1} & x \neq \bar{x}\\
\end{matrix} \tag{2.1}$

where $\bar{x}$ is the mean of the population, $x$ is the value being compared to the sample, $s$ is the standard devation of the sample, and there are $n$ observations in the sample.

## 2.1 Test Statistic

The test statistic for the case I t test is given as
$t = \frac{(\bar{x} - x)\sqrt{n}}{s} \tag{2.2}$
The probability distribution follows a T distribution with $n-1$ degrees of freedom, where $n$ is the number of observations in the sample.


For the emperical test, we will use the `scipy.stats.ttest_1samp` function, which returns a p value.

In [ ]:
def emp_ttest_ind(sample1, sample2):
    return scipy.stats.ttest_ind(sample1, sample2)[1]

## 2.2 Nonparametric Test

In [ ]:
def emp_ttest_ind_nonpar(sample1, sample2):
    return scipy.stats.mannwhitneyu(sample1, sample2)[1]

## 2.3 Effect Size

The effect size for a one-sample t test is Cohen's d, given by equation (2.3).
$\begin{align*}
d = \frac{\bar{x} - x_{0}}{s}
\end{align*}\tag{2.3}$
For a two-tailed test, we can treat this as an absloute magnitude, and then test $t > (1 - \alpha/2)$.

In [ ]:
def cohen_ttest_ind(sample1, sample2):
    x1, s1, n1 = sample1.mean(), sample1.std(), len(sample1)
    x2, s2, n2 = sample2.mean(), sample2.std(), len(sample2)
    
    spool = np.sqrt(((n1-1) * np.square(s1) + (n2 - 1) * np.square(s2)) / (n1 + n2 - 2))
    return np.absolute(x1 - x2) / spool

## 2.5 Simulation Parameters

We'll test the set of hypotheses,

$\begin{matrix}
\textbf{H}_{0} & 0 = \bar{x} \\
\textbf{H}_{1} & 0 \neq \bar{x}\\
\end{matrix} \tag{2.5}$

where $\bar{x}$ is the mean of a sample drawn from a population.

We'll test the means between 2 and 10 with standard deviations between 5 and 15, and sample sizes between 60 and 100 observations.

In [ ]:
def ttest_ind_simulate(mu_lim, sigma_lim, count_lims, bench_lims, diff_lim):
    # Gets the distribution parameters
    mu = np.random.uniform(*mu_lim, size=2)
    offset = np.random.uniform(*diff_lim)
    sigma = np.random.uniform(*sigma_lim, size=2)
    n = np.random.randint(*count_lims)
    m = np.random.randint(*bench_lims)
    
    #     Draws a sample that fits the parameters
    sample1 = np.hstack((np.random.normal(mu[0], sigma[0], n - m), 
                         np.random.normal(mu[1], sigma[1], m)))
    sample2 = np.hstack((np.random.normal(mu[0], sigma[0], n - m), 
                         np.random.normal(mu[1], sigma[1], m))) + offset

    return [mu, sigma, offset, n, m], [sample1, sample2]

In [ ]:
distributions['ttest_ind'] = {'emp_test_fun': emp_ttest_ind,
                            'non_test_fun': emp_ttest_ind_nonpar,
                            'trad_test_fun': calc_ttest_ind,
                            'sim_function': ttest_ind_simulate,
                            'sim_parameters': [[1, 6.], [1., 5.], [125., 126.]],
                            'other_sim_parameters': {'bench_lims': [25, 75], 'diff_lim': [0.5, 7.5]},
                            'test_parameters': {},
                            'emp_parameters': {},
                            'effect_size': cohen_ttest_ind,
                             }

# 6. Building Simulations

We will now build the simulations and look at the results.

In [ ]:
num_rounds

In [ ]:
%%time

# for k, params in distributions.iteritems():
for k, params in [('ttest_ind', distributions['ttest_ind'])]:
    params['pop_params'] = []
    params['sample'] = []
    params['effs'] = []
    params['test'] = []
    params['trad'] = []
    params['empp'] = []
    params['empn'] = []
    params['extp'] = []
    params['extn'] = []
    
    for i in xrange(10):
        # Calculates the samples
        pop_params, samples = params['sim_function'](*params['sim_parameters'], **params['other_sim_parameters'])
        params['pop_params'].append(pop_params)
       
        # Calculates the traditional power
        test_power = params['trad_test_fun'](*samples, counts=counts, **params['test_parameters'])
        params['test'].append(test_power)
        
        # Calculates Cohen's effect
        if 'effect_size' in params:
            cohen_eff = params['effect_size'](*samples)
        else:
            cohen_eff = None
        
        # Extrapolates the effect from the test
        test_eff = ap.z_effect(counts, test_power)
        
        trad_power = ap.z_power(counts, test_eff.mean())
        params['trad'].append(trad_power)
        
         # Calculates the emprical power using a nonparametric test
        nempr_power, nempr_counts = subsample_power(lambda x: params['non_test_fun'](*x, **params['test_parameters']),
                                                  samples,
                                                  min_counts=5,
                                                  max_counts=65,
                                                  counts_interval=10,
                                                  num_runs=5,
                                                  num_iter=1000)
        # Calculates the nonparametric effect
        nonp_eff = ap.z_effect(nempr_counts, nempr_power)
        next_power = ap.z_power(counts, nonp_eff.mean())
        params['empn'].append(nempr_power)
        params['extn'].append(next_power)    

        # Updates the information
        params['effs'].append([cohen_eff, test_eff, nonp_eff])
        
    distributions[k] = params
    print k

In [ ]:
# fig1 = plt.figure()
# fig1.set_size_inches((4, 4))

samples = params['sample']

effs = params['effs']

test = np.vstack(params['test'])
trad = np.vstack(params['trad'])
empn = np.vstack([d.mean(0) for d in params['empn']])
emps = np.vstack([d.std(0) / np.sqrt(len(d)) * scipy.stats.t.ppf(0.975, len(d) - 1) 
                  for d in params['empn']])
extn = np.vstack(params['extn'])
extp = np.vstack(params['extp'])

In [ ]:
cohen_d = np.hstack(zip(*effs)[0])
print cohen_d.min()
print cohen_d.max()

In [ ]:
from matplotlib import cm

means = np.array([np.mean(e) for e in zip(*effs)[2]])
means_order = np.argsort(means)

num_colors=len(means_order) # cats is a vector of the number of groups
blues = cm.get_cmap('Blues') # Colormap name
greens = cm.get_cmap('Greens')
reds = cm.get_cmap('Reds')
purples = cm.get_cmap('Purples')
new_blues = np.array([list(blues((1.*i)/(num_colors))) for i in xrange(num_colors)])
new_green = np.array([list(greens((1.*i)/(num_colors))) for i in xrange(num_colors)])
new_reds = np.array([list(reds((1.*i)/(num_colors))) for i in xrange(num_colors)])
new_purples = np.array([list(purples((1.*i)/(num_colors))) for i in xrange(num_colors)])

In [ ]:
# new_blues = [[0.5, 0.5, 0.5]] * num_colors
# new_green = [[0.5, 0.5, 0.5]] * num_colors
# new_reds = [[0.5, 0.5, 0.5]] * num_colors
# new_purples = [[0.5, 0.5, 0.5]] * num_colors

In [ ]:
test[0]

In [ ]:
empn.shape

In [ ]:
fig = plt.figure()

ax2 = fig.add_subplot(2, 2, 1)
ax3 = fig.add_subplot(2, 2, 2)
ax6 = fig.add_subplot(2, 2, 3)
ax7 = fig.add_subplot(2, 2, 4)


# ax5.fill_between([0, 1], [-0.15, -0.15], [0.15, 0.15], color=[0.75, 0.75, 0.75], alpha=0.5)
ax6.fill_between([0, 1], [-0.15, -0.15], [0.15, 0.15], color=[0.75, 0.75, 0.75], alpha=0.5)
ax7.fill_between([0, 1], [-0.15, -0.15], [0.15, 0.15], color=[0.75, 0.75, 0.75], alpha=0.5)
# ax8.fill_between([0, 1], [-0.15, -0.15], [0.15, 0.15], color=[0.75, 0.75, 0.75], alpha=0.5)



# for idx in xrange(len(means_order)):
for idx in xrange(100):
    loc = means_order[idx]
    cohen_eff, test_eff, nonp_eff, nonf_eff, parz_eff, parf_eff = effs[idx] 
    # Sets up the power based on the traditional effect
    
#     # Sets up the power based on the emprical nonparametric test
    ax2.errorbar(test[idx], empn[idx], emps[idx], 
                 linestyle='none', color=new_reds[loc]
                 )
    ax2.plot(test[idx], empn[idx], 'o', mfc='none', mec=new_reds[loc])
    ax6.errorbar(test[idx],
                 (empn[idx] - test[idx]) / test[idx],
                 emps[idx] / test[idx],
                 linestyle='none', color=new_reds[idx]
                )
    ax6.plot(test[idx], (empn[idx] - test[idx]) / test[idx], 'o', mfc='none', mec=new_reds[idx])
    
    # Sets up the power based on the extrapolated nonparametric test
    n_pwr = np.array([ap.z_power(counts, e) for e in nonp_eff])
    ax3.errorbar(test[idx], n_pwr.mean(0), 
                 n_pwr.std(0) / np.sqrt(len(nonp_eff)) * scipy.stats.t.ppf(0.975, len(nonp_eff)-1),
                 linestyle='none', color=new_blues[loc])
    ax3.plot(test[idx], n_pwr.mean(0), 'o', mfc='none', mec=new_blues[loc])
    ax7.errorbar(test[idx], (n_pwr.mean(0) - test[idx]) / test[idx],
                 n_pwr.std(0) / np.sqrt(len(nonp_eff)) * scipy.stats.t.ppf(0.975, len(nonp_eff)-1),
                 linestyle='none', color=new_blues[loc]
                 )
    ax7.plot(test[idx], (n_pwr.mean(0) - test[idx])/test[idx], 'o', mfc='none', mec=new_blues[loc])
    
for i, ax in enumerate(fig.axes):
    ax.set_xticks(np.arange(0, 1.1, 0.25))
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.get_xaxis().set_tick_params(direction='out')
    ax.get_yaxis().set_tick_params(direction='out')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    if i < 2:
        ax.set_yticks(np.arange(0, 1.1, 0.25))
        ax.set_yticklabels('')
        ax.set_xticklabels('')
        ax.plot([0, 1], [0, 1], 'k-', linewidth=0.5)
    else:
        ax.set_ylim([-1.5, 1.5])
        ax.set_yticks(np.arange(-1.5, 1.6, 0.5))
        ax.set_yticklabels('')
        ax.set_xticklabels(np.arange(0, 1.1, 0.25), size=11)
        ax.plot([0, 1], [0, 0], 'k-', linewidth=0.5)
        
ax2.set_yticklabels(np.arange(0, 1.1, 0.25), size=11)
ax6.set_yticklabels(np.arange(-1.5, 1.6, 0.5), size=11)
# ax5.set_yticklabels(np.arange(-1, 1.1, 0.5), size=11)

l = ax6.set_xlabel('Parametric power', size=12)
l.set_position((1.1, 0.5))
ax2.set_ylabel('Nonparametric Power', size=12)
ax6.set_ylabel('Relative Difference', size=12)

ax2.set_title('Emprical Power', size=12)
ax3.set_title('Extrapolated Power', size=12)

fig.set_size_inches((4, 4))


In [ ]:
nonpara = [(x.mean(), x.std(), len(x)) for x in zip(*effs)[2]]

In [ ]:
plt.plot(zip(*effs)[0], zip(*nonpara)[0], 'o')

In [ ]:
d = np.array(zip(*effs)[0])
e = np.array(zip(*nonpara)[0])

In [ ]:
idx = 0
e[idx], d[idx]

In [ ]:

plt.plot(np.array(zip(*effs)[0]), empn[:, 1], 'o')
for idx in xrange(len(effs)):
    if ((e[idx]/d[idx] > 1.5) or (d[idx] / e[idx] > 1.5)) and ((d[idx] > 0.15) and (e[idx] > 0.15)):
        plt.plot(d[idx], empn[idx, 1], 'go')
plt.plot(np.array(zip(*nonpara)[0]), empn[:, 1], 'r.')

In [ ]:

for idx in xrange(len(effs)):
    if ((e[idx]/d[idx] > 1.5) or (d[idx] / e[idx] > 1.5)) and ((d[idx] > 0.1) and (e[idx] > 0.1)):
        s1, s2 = distributions['ttest_ind']['sample'][idx]
        d1, d2 = distributions['ttest_ind']['pop_params'][idx][1]
        

        print '%i\t%1.3f\t%1.3f\t%1.3f\t%1.3f\t%1.1e\t%1.1e\t%1.1f\t%1.1f' % (idx, d[idx], empn[idx, 1], e[idx], e[idx]/d[idx], emp_ttest_ind_nonpar(s1, s2), emp_ttest_ind(s1, s2), d1, d2)


        fig = plt.figure()
        fig.set_size_inches((12, 4))
        ax1 = fig.add_subplot(2, 2, 1)
        ax2 = fig.add_subplot(2, 2, 3)
        ax3 = fig.add_subplot(1, 2, 2)

        s1, s2 = distributions['ttest_ind']['sample'][idx]
        xbar1, std1 = s1.mean(), s1.std()
        xabr2, std2 = s2.mean(), s2.std()
        # [shapes, diffs, scales, ns, ms] = distributions['ttest_ind']['pop_params'][idx]

        ax1.hist(s1, np.arange(0, 75, 2.5), facecolor='none', normed=True)
        ax1.plot(np.arange(0, 75, 1), scipy.stats.norm.pdf(np.arange(0, 75, 1), s1.mean(), s1.std()), 'r-')

        ax2.hist(s2, np.arange(0, 75, 2.5), facecolor='none', normed=True)
        ax2.plot(np.arange(0, 75, 1), scipy.stats.norm.pdf(np.arange(0, 75, 1), s1.mean(), s1.std()), 'r-')

        ax3.plot(test[idx], empn[idx], 'o')
        ax3.plot(test[idx], extn[idx], 's')
        ax3.plot([0, 1], [0, 1], 'k-')
        
        fig.savefig('/Users/jwdebelius/Desktop/power_check/%i.png' % idx)
        plt.close(fig)

In [ ]:
[shapes, diffs, scales, ns, ms] = distributions['ttest_ind']['pop_params'][idx]

In [ ]:
sample1 = np.hstack((np.random.gamma(shapes[0], scales[0], ns[0] - ms[0]), 
                         np.random.gamma(shapes[0] + diffs[0], scales[1], ms[0])))
    sample2 = np.hstack((np.random.gamma(shapes[1], scales[2], ns[1] - ms[1]), 
                         np.random.gamma(shapes[1] + diffs[1], scales[3], ms[1])))

In [ ]:
distributions['ttest_ind'].keys()

In [ ]:
#     ax5.plot(distributions['ttest_1']['test'][idx],
#              (-distributions['ttest_1']['test'][idx] +
#               distributions['ttest_1']['trad'][idx]) /
#              distributions['ttest_1']['test'][idx],
#             '.', color=new_green[loc])
#     ax1.set_axis_bgcolor('k')
#     ax2.plot(distributions['ttest_1']['test'][idx],
#              np.mean(distributions['ttest_1']['empn'][idx], 0),
#              '.', color=new_blues[loc])
# #     ax2.set_axis_bgcolor('k')
#     ax6.plot(distributions['ttest_1']['test'][idx],
#              (-distributions['ttest_1']['test'][idx] +
#               np.mean(distributions['ttest_1']['empn'][idx], 0)) /
#              distributions['ttest_1']['test'][idx],
#             '.', color=new_blues[loc])
    
#     ax3.plot(distributions['ttest_1']['test'][idx],
#              distributions['ttest_1']['extn'][idx],
#              '.', color=new_reds[loc])
    
#     ax7.plot(distributions['ttest_1']['test'][idx],
#              (-distributions['ttest_1']['test'][idx] +
#               distributions['ttest_1']['extn'][idx]) /
#              distributions['ttest_1']['test'][idx],
#             '.', color=new_reds[loc])
    
# # #     ax3.set_axis_bgcolor('k')
#     ax4.plot(distributions['ttest_1']['extn'][idx],
#              np.mean(distributions['ttest_1']['empn'][idx], 0),
#              '.', color=new_purples[loc])
    
#     ax8.plot(distributions['ttest_1']['extn'][idx],
#              (np.mean(distributions['ttest_1']['empn'][idx], 0) -
#               distributions['ttest_1']['extn'][idx]) /
#               distributions['ttest_1']['extn'][idx],
#             '.', color=new_purples[loc])
    
# for i, ax in enumerate(fig.axes):
#     ax.set_xticks(np.arange(0, 1.1, 0.25))
#     if i < 4:
#         ax.set_yticks(np.arange(0, 1.1, 0.25))
#         ax.set_yticklabels('')
#         ax.set_xticklabels('')
#         ax.plot([0, 1], [0, 1], 'k-', linewidth=0.5)
#     else:
#         ax.set_ylim([-1, 1])
#         ax.set_yticks(np.arange(-1, 1.1, 0.5))
#         ax.set_yticklabels('')
#         ax.set_xticklabels(np.arange(0, 1.1, 0.25), size=11)
#         ax.plot([0, 1], [0, 0], 'k-', linewidth=0.5)
        
# ax1.set_yticklabels(np.arange(0, 1.1, 0.25), size=11)
# ax5.set_yticklabels(np.arange(-1, 1.1, 0.5), size=11)


fig.set_size_inches((8, 4))

In [ ]:
d, te, xe, fe = zip(*effs)

In [ ]:
d

In [ ]:
xem = np.array([x.mean() for x in xe])

In [ ]:
xem

In [ ]:
plt.plot(d, xem / np.sqrt(2), 'x')
plt.plot([0, 2.5], [0, 2.5], 'k-')

In [ ]:
means[means_order] / np.sqrt(2)

In [ ]:
np.mean([ap.z_power(counts, e) for e in zip(*effs)[1][0]], 0)

In [ ]:
trad_mean = np.concatenate([ap.z_power(counts, np.mean(eff)) for eff in zip(*effs)[1]])
trad_conf = np.concatenate([(ap.z_power(counts, np.mean(eff)) + np.std(eff) / np.sqrt(len(eff)) * scipy.stats.t.ppf(0.975, len(eff) - 1)) - 
                            ap.z_power(counts, np.mean(eff)) for eff in zip(*effs)[1]])

In [ ]:
tst_trd_m = np.concatenate([np

In [ ]:
check = [d.mean(0) for d in distributions['ttest_1']['empn']]

In [ ]:
np.hstack(check)

In [ ]:
empn_mean = [d.mean(0) for d in distributions['ttest_1']['empn']]
empn_conf = np.concatenate([d.std(0) / np.sqrt(5) * scipy.stats.t.ppf(0.975, 2) for d in distributions['ttest_1']['empn']])

emntrd_rt = np.concatenate([np.mean((np.vstack([t] * 3) - d) / np.vstack([t] * 3), 0) / np.sqrt(len(d))
                           for (t, d) in zip(*(distributions['ttest_1']['trad'], distributions['ttest_1']['empn']))])
emntrd_ci = np.concatenate([np.std((np.vstack([t] * 3) - d) / np.vstack([t] * 3), 0) / np.sqrt(len(d)) * scipy.stats.t.ppf(0.975, len(d)-1) 
                            for (t, d) in zip(*(distributions['ttest_1']['trad'], distributions['ttest_1']['empn']))])

In [ ]:
extn_mean = np.concatenate([ap.z_power(counts, np.mean(eff)) for eff in zip(*effs)[3]])
extn_conf = np.concatenate([ap.z_power(counts, np.mean(eff) + np.std(eff) / np.sqrt(len(eff)) * scipy.stats.t.ppf(0.975, len(eff) - 1)) - 
                            ap.z_power(counts, np.mean(eff)) for eff in zip(*effs)[3]])
exntrd_r = []
exntrd_c = []
for i, (t, eff) in enumerate(zip(*(distributions['ttest_1']['trad'], distributions['ttest_1']['effs']))):
    if len(eff[3]) > 0:
        exntrd_r.append(np.mean([(ap.z_power(counts, e) - t) / t for e in eff[3]], 0))
        exntrd_c.append(np.std([(ap.z_power(counts, e) - t) / t for e in eff[3]], 0) / np.sqrt(len(eff[3])) * scipy.stats.t.ppf(0.975, len(eff[3] -1)))
    else:
        exntrd_r.append(np.nan*np.ones(counts.shape))
        exntrd_c.append(np.nan*np.ones(counts.shape))
    
exntrd_r = np.concatenate(exntrd_r)
exntrd_c = np.concatenate(exntrd_c)

In [ ]:
test.shape

In [ ]:
empn_mean.shape

In [ ]:
fig2 = plt.figure()

ax1 = fig2.add_subplot(2, 3, 1)
ax2 = fig2.add_subplot(2, 3, 2)
ax3 = fig2.add_subplot(2, 3, 3)
ax4 = fig2.add_subplot(2, 3, 4)
ax5 = fig2.add_subplot(2, 3, 5)
ax6 = fig2.add_subplot(2, 3, 6)

ax1.errorbar(test, trad_mean, trad_conf, marker='', linestyle='none', color='#bdbdbd')
ax1.plot(test, trad_mean, 'o', mfc='none', ms=4, alpha=0.2, mec='#252525')

ax3.plot(test, empn_mean)

# ax3.errorbar(test, empn_mean, empn_conf, marker='', linestyle='none', color='#fb9a99')
# ax3.plot(test, empn_mean, 'o', mfc='none', ms=4, alpha=0.2, mec='#e31a1c')

In [ ]:
ax2.errorbar(trad, empn_mean, yerr=empn_conf, marker='', linestyle='none', color='#a6cee3')
ax2.plot(trad, empn_mean, 'o', mec='#1f78b4', mfc='none', markersize=4, alpha=0.2)

ax2.set_xlim([-0.05, 1.05])
ax2.set_ylim([-0.05, 1.1])
ax2.plot([-0.1, 1.1], [-0.1, 1.1], 'k-')
ax2.set_xticks(np.arange(0, 1.1, 0.25))
ax2.set_yticks(np.arange(0, 1.1, 0.25))
ax2.set_xticklabels('')
ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.yaxis.set_ticks_position('left')
ax2.xaxis.set_ticks_position('bottom')
ax2.get_xaxis().set_tick_params(direction='out')
ax2.get_yaxis().set_tick_params(direction='out')
ax2.set_yticklabels(ax2.get_yticks(), size=10)
ax2.set_ylabel('Emperical Power', size=12)
ax2.text(0.05, 0.9, '(A)', size=15)

ax4.errorbar(trad, emntrd_rt, yerr=emntrd_ci, marker='', linestyle='none', color='#a6cee3')
ax4.plot(trad, emntrd_rt, 'o', mec='#1f78b4', mfc='none', markersize=4, alpha=0.2)
ax4.plot([0, 1], [0, 0], 'k-')
ax4.set_ylim([-1, 1])
ax4.set_xticks(np.arange(0, 1.1, 0.25))
ax4.set_yticks(np.arange(-1, 1.1, 0.5))
# ax4.set_xticklabels('')
ax4.set_yticklabels(ax4.get_yticks(), size=10)
ax4.set_xticklabels(ax4.get_xticks(), size=10)
ax4.spines['right'].set_visible(False)
ax4.spines['top'].set_visible(False)
ax4.yaxis.set_ticks_position('left')
ax4.xaxis.set_ticks_position('bottom')
ax4.get_xaxis().set_tick_params(direction='out')
ax4.get_yaxis().set_tick_params(direction='out')
ax4.set_ylabel('Relative Difference', size=12)
ax4.text(0.05, 0.8, '(C)', size=15)

ax3.errorbar(trad, extn_mean, yerr=extn_conf, marker='', linestyle='none', color='#fb9a99')
ax3.plot(trad, extn_mean, 'o', mec='#e31a1c', mfc='none', markersize=4)

ax3.plot([0, 1], [0, 1], 'k-')
ax3.set_xticks(np.arange(0, 1.1, 0.25))
ax3.set_yticks(np.arange(0, 1.1, 0.25))
ax3.set_yticklabels('')
ax3.set_xticklabels('')
ax3.spines['right'].set_visible(False)
ax3.spines['top'].set_visible(False)
ax3.yaxis.set_ticks_position('left')
ax3.xaxis.set_ticks_position('bottom')
ax3.get_xaxis().set_tick_params(direction='out')
ax3.get_yaxis().set_tick_params(direction='out')
ax3.text(0.05, 0.9, '(B)', size=15)


ax5.errorbar(trad, exntrd_r, yerr=exntrd_c, marker='', linestyle='none', color='#fb9a99')
ax5.plot(trad, exntrd_r, 'o', mec='#e31a1c', mfc='none', ms=4)
ax5.plot([0, 1], [0, 0], 'k-')
ax5.set_xticks(np.arange(0, 1.1, 0.25))
ax5.set_ylim([-1, 1])
ax5.set_yticklabels('')
ax5.set_xticklabels(ax5.get_xticks(), size=10)
ax5.spines['right'].set_visible(False)
ax5.spines['top'].set_visible(False)
ax5.yaxis.set_ticks_position('left')
ax5.xaxis.set_ticks_position('bottom')
ax5.get_xaxis().set_tick_params(direction='out')
ax5.get_yaxis().set_tick_params(direction='out')
ax5.text(0.05, 0.8, '(D)', size=15)
l = ax5.set_xlabel('Traditional Power', size=12)
l.set_position((-0.1, 1))

fig2.set_size_inches((4, 4))

In [ ]:
print effs[0][1].mean(), effs[0][1].std()
print effs[0][3].mean(), effs[0][3].std()

In [ ]:
trad_eff_mean = np.array([np.mean(e) for e in zip(*effs)[1]])
trad_eff_stdv = np.array([np.std(e) for e in zip(*effs)[1]])

extn_eff_mean = np.array([np.mean(e) for e in zip(*effs)[3]])
extn_eff_stdv = np.array([np.std(e) for e in zip(*effs)[3]])

In [ ]:
trad_eff_mean.shape

In [ ]:
extn_eff_mean.shape

In [ ]:
plt.errorbar(trad_eff_mean, extn_eff_mean, trad_eff_stdv, extn_eff_stdv, mfc='none', linewidth=1, linestyle='none')